In [39]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
import plotly.express as px
import psycopg2 as pg
import sqlalchemy
import matplotlib as plt
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sklearn as skl
import tensorflow as tf

In [40]:
#Read vaersdata dataframe from postgreSQL 

connection = pg.connect(host='team-11.ch3wmigorcf0.us-east-2.rds.amazonaws.com', dbname="postgres", user="postgres", password='UTB00tcamp')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
VAERS_data_df = pd.read_sql_query('select * from public."vaersdata"',con=connection)
VAERS_data_df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,None,Right side of epiglottis swelled up and hinder...,0,...,None,None,None,None,2,01/01/2021,None,Y,None,Pcn and bee venom
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,None,"""Approximately 30 min post vaccination adminis...",0,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,None,None,2,01/01/2021,None,Y,None,"""""""Dairy"""""""
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,None,"About 15 minutes after receiving the vaccine, ...",0,...,None,None,None,None,2,01/01/2021,None,None,Y,Shellfish
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,None,"extreme fatigue, dizziness,. could not lift my...",0,...,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",None,2,01/01/2021,None,None,None,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,None,"Injection site swelling, redness, warm to the ...",0,...,Na,None,None,None,2,01/01/2021,None,None,None,Na


In [41]:
VAERS_data_df=VAERS_data_df.drop(['RECVDATE', 'RPT_DATE', 'SYMPTOM_TEXT', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,None,None,None,None,Pcn and bee venom
1,916601,73.0,F,0,None,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""""""Dairy"""""""
2,916602,23.0,F,0,None,None,None,None,Shellfish
3,916603,58.0,F,0,None,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,None,Na,Na,None,Na


In [42]:
#Transform String Columns
def change_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_data_df["DIED"]= VAERS_data_df["DIED"].apply(change_string)
VAERS_data_df.head(20)

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,None,None,None,None,Pcn and bee venom
1,916601,73.0,F,0,None,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""""""Dairy"""""""
2,916602,23.0,F,0,None,None,None,None,Shellfish
3,916603,58.0,F,0,None,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,None,Na,Na,None,Na
5,916606,44.0,F,0,None,None,None,None,iodine (shellfish) has epipen
6,916607,50.0,M,0,None,"Amlodipine, Ambien, Benicar/HCTZ, Invokana, Me...",None,"High blood pressure, high cholesterol, sleep a...",Penicillin
7,916608,33.0,M,0,None,None,None,None,None
8,916609,71.0,F,0,None,Medication Summary 1/1/21 Name of Medica...,none,"Hashimoto's thyroiditis, Hypertension, depression","Sulfa antibiotics, azithromycin, adhesive in ..."
9,916610,18.0,F,0,None,None,None,None,jackfruit


In [43]:
#Transform String Columns
def change_string1(hospital):
    if hospital == "Y":
        return 1
    else:
        return 0 

VAERS_data_df["HOSPITAL"]= VAERS_data_df["HOSPITAL"].apply(change_string1)
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,0,None,None,None,Pcn and bee venom
1,916601,73.0,F,0,0,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""""""Dairy"""""""
2,916602,23.0,F,0,0,None,None,None,Shellfish
3,916603,58.0,F,0,0,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,0,Na,Na,None,Na


In [44]:
#Transform String Columns
VAERS_data_df['OTHER_MEDS'] = VAERS_data_df['OTHER_MEDS'].notnull().copy()


In [45]:
VAERS_data_df['OTHER_MEDS']=VAERS_data_df['OTHER_MEDS'].map({False: 0, True: 1}).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,0,1,None,None,Pcn and bee venom
1,916601,73.0,F,0,0,1,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""""""Dairy"""""""
2,916602,23.0,F,0,0,1,None,None,Shellfish
3,916603,58.0,F,0,0,1,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,0,1,Na,None,Na


In [46]:
#Transform String Columns
VAERS_data_df['CUR_ILL'] = VAERS_data_df['CUR_ILL'].notnull().copy()
VAERS_data_df['CUR_ILL']=VAERS_data_df['CUR_ILL'].map({False: 0, True: 1}).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,0,1,1,None,Pcn and bee venom
1,916601,73.0,F,0,0,1,1,Patient residing at nursing facility. See pati...,"""""""Dairy"""""""
2,916602,23.0,F,0,0,1,1,None,Shellfish
3,916603,58.0,F,0,0,1,1,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,0,1,1,None,Na


In [47]:
#Transform String Columns
VAERS_data_df['HISTORY'] = VAERS_data_df['HISTORY'].notnull().copy()
VAERS_data_df['HISTORY']=VAERS_data_df['HISTORY'].map({False: 0, True: 1}).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,0,1,1,1,Pcn and bee venom
1,916601,73.0,F,0,0,1,1,1,"""""""Dairy"""""""
2,916602,23.0,F,0,0,1,1,1,Shellfish
3,916603,58.0,F,0,0,1,1,1,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,47.0,F,0,0,1,1,0,Na


In [48]:
#Transform String Columns
VAERS_data_df['ALLERGIES'] = VAERS_data_df['ALLERGIES'].notnull().copy()
VAERS_data_df['ALLERGIES']=VAERS_data_df['ALLERGIES'].map({False: 0, True: 1}).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,F,0,0,1,1,1,1
1,916601,73.0,F,0,0,1,1,1,1
2,916602,23.0,F,0,0,1,1,1,1
3,916603,58.0,F,0,0,1,1,1,1
4,916604,47.0,F,0,0,1,1,0,1


In [49]:
#Transform String Column "SEX"
def no_string(gender):
    if gender == "M":
        return 1
    else:
        return 0 

VAERS_data_df["SEX"]= VAERS_data_df["SEX"].apply(no_string)
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916600,33.0,0,0,0,1,1,1,1
1,916601,73.0,0,0,0,1,1,1,1
2,916602,23.0,0,0,0,1,1,1,1
3,916603,58.0,0,0,0,1,1,1,1
4,916604,47.0,0,0,0,1,1,0,1


In [50]:
VAERS_data_df = pd.get_dummies(VAERS_data_df, prefix=["GENDER"], columns= ["SEX"]).copy()
VAERS_data_df.head()

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
0,916600,33.0,0,0,1,1,1,1,1,0
1,916601,73.0,0,0,1,1,1,1,1,0
2,916602,23.0,0,0,1,1,1,1,1,0
3,916603,58.0,0,0,1,1,1,1,1,0
4,916604,47.0,0,0,1,1,0,1,1,0


In [51]:
VAERS_data_df.AGE_YRS=VAERS_data_df.AGE_YRS.fillna(0)
VAERS_data_df.AGE_YRS=VAERS_data_df.AGE_YRS.astype(int)

In [52]:
VAERS_data_df.dtypes

VAERS_ID      int64
AGE_YRS       int32
DIED          int64
HOSPITAL      int64
OTHER_MEDS    int64
CUR_ILL       int64
HISTORY       int64
ALLERGIES     int64
GENDER_0      uint8
GENDER_1      uint8
dtype: object

In [53]:
VAERS_data_df.head(10)

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
0,916600,33,0,0,1,1,1,1,1,0
1,916601,73,0,0,1,1,1,1,1,0
2,916602,23,0,0,1,1,1,1,1,0
3,916603,58,0,0,1,1,1,1,1,0
4,916604,47,0,0,1,1,0,1,1,0
5,916606,44,0,0,0,0,0,1,1,0
6,916607,50,0,0,1,1,1,1,0,1
7,916608,33,0,0,1,1,1,1,0,1
8,916609,71,0,0,1,1,1,1,1,0
9,916610,18,0,0,0,0,0,1,1,0
